In [ ]:
import json
from config import settings
import os


import polars as pl
from polars import col as c
from polars import selectors as cs
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp

# from distflow_algorithm import DistFlow
import time
import pandapower as pp
import pandapower.networks as pn
from polars_function import (
    get_transfo_admittance,
    get_transfo_impedance,
    get_transfo_conductance,
    get_transfo_imaginary_component,
)

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import get_all_edge_data, generate_shortest_path_length_matrix
import graphblas as gb
from utility.parser_utility import duckdb_to_changes_schema

from twindigrid_sql.entries.equipment_class import (
    TRANSFORMER,
    BRANCH,
    SWITCH,
    EXTERNAL_NETWORK,
)

from distflow_schema import DistFlowSchema

from state_estimation.matrix_generators import generate_full_jacobian_matrix

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

## Import Schema from duckdb

In [ ]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
changes_schema = duckdb_to_changes_schema(file_path=file_names["duckdb_output"])
list(changes_schema.__dict__.keys())

## Create resource data

In [ ]:
TRANSFORMER
BRANCH
SWITCH
EXTERNAL_NETWORK

In [ ]:
resource_data = changes_schema.resource.filter(
    (c("concrete_class") == BRANCH)
    | (c("concrete_class") == SWITCH)
    | (c("concrete_class") == TRANSFORMER)
    | (c("concrete_class") == EXTERNAL_NETWORK)
).select(
    c("uuid"), c("dso_code").alias("element_id"), c("concrete_class").alias("type")
)

## Add row index to connectivity_node to give the number of node
## Join connectivity_node with connectivity to get the eq_fk
## Create the connectivity_node dictionnary with side+eq_fk as key and node index as value
cn_mapping: dict[float, str] = pl_to_dict(
    changes_schema.connectivity_node.with_row_index()
    .join(changes_schema.connectivity, left_on="uuid", right_on="cn_fk", how="left")
    .select((c("side") + c("eq_fk")).alias("eq_fk_side"), c("index"))
)

## Add node from and node to for each edge with side+eq_fk
resource_data = resource_data.with_columns(
    ("t1" + c("uuid"))
    .replace_strict(cn_mapping, default=None)
    .alias(
        "u_of_edge"
    ),  # Replace side+eq_fk with node number from connectivity for equipment
    ("t2" + c("uuid")).replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)
# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]

# Add branch parameter to line_data
resource_data = resource_data.join(
    branch, left_on="uuid", right_on="eq_fk", how="left"
).drop("uuid_right")
## Add n_tranfo to 1, useless, because automatically added by add_table from class DistFlowSchema and by default value is 1
# resource_data = resource_data.with_columns(pl.lit(1).alias("n_tranfo"))

## Search slack node id for DisFlowSchema
slack_node_id = resource_data.filter(c("type") == EXTERNAL_NETWORK)[
    "u_of_edge"
].item()

## Remove the external network from the resource_data
resource_data = resource_data.filter(c("type") != EXTERNAL_NETWORK)

## Transform resource_data in pu

In [ ]:
## Transfo in pu
s_base = 1e6  # VA -> 1 MVA for distribution grid

u_b = changes_schema.base_voltage["nominal_voltage"].item()  # V
i_b = s_base / (3**0.5 * U_b)  # A
z_b = u_b**2 / s_base  # Ohm
b_b = 1 / Z_b  # S


pu_base_changes_schema = {
    "U_b": u_b,
    "I_b": i_b,
    "Z_b": z_b,
    "B_b": b_b,
    "S_base": s_base,
}

resource_data_pu = resource_data.with_columns(
    (c("g") * pu_base_changes_schema["Z_b"]).alias("g_pu"),
    (c("r") / pu_base_changes_schema["Z_b"]).alias("r_pu"),
    (c("x") / pu_base_changes_schema["Z_b"]).alias("x_pu"),
    (c("b") / pu_base_changes_schema["B_b"]).alias("b_pu"),
).drop(["r", "x", "b", "g"])

## Check if B is negativ for branch and positiv for trafo

In [ ]:
## If type is transformer, then b_pu is positive, otherwise negative
resource_data_pu = resource_data_pu.with_columns(
    pl.when(c("type") == TRANSFORMER)
    .then(c("b_pu").abs())
    .otherwise(
        pl.when((c("type") == BRANCH) | (c("type") == SWITCH)).then(
            c("b_pu").abs().neg()
        )
    )
)

In [ ]:
## Create the DistFlowSchema instance
distflow_schema = DistFlowSchema()
## Add edge_data table to the DistFlowSchema instance
distflow_schema = distflow_schema.add_table(edge_data=resource_data_pu)

In [ ]:
generate_full_jacobian_matrix(distflow_schema, slack_node_id)

In [ ]:
## u, v and unpivot
# line_data.pivot(index="eq_class", on="side", values="node_number")
## add r,x,b,g in pu

## import type (switch, branch, transformer) from table

## add n_trafo
# distflow_data = line_data.filter().pivot.....

In [ ]:
# edge_data: pl.DataFrame = line_data
# u_of_edge: int = pt.Field(dtype=pl.Int32)
# v_of_edge: int = pt.Field(dtype=pl.Int32)
# r_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# x_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# b_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# g_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# n_transfo: float = pt.Field(dtype=pl.Float64, default=1.0)
# type: TYPES = pt.Field(dtype=pl.Utf8, constraints=literal_constraint(pt.field, TYPES))

## Filter data and create distflowschema

In [ ]:
# edge_data = 1
# test = DistFlowSchema(edge_data)

In [ ]:
# edge_data = test.edge_data
changes_schema.resource

switch

branch

branch_parameter_event

In [ ]:
line_data = changes_schema.resource.filter(
    (c("concrete_class") == BRANCH) | (c("concrete_class") == SWITCH)
).select(c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type"))
print(line_data)
cn_mapping = pl_to_dict(
    changes_schema.connectivity.select(
        pl.concat_str("eq_fk", "side").alias("id"), "uuid"
    )
)
## Add uuid of t1 and t2 for resource
line_data = line_data.with_columns(
    (c("uuid") + "t1")
    .replace_strict(cn_mapping, default=None)
    .alias("u_of_edge"),  # Replace eq_fk with uuid from connectivity for branch
    (c("uuid") + "t2").replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)

# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]
# changes_schema.branch_parameter_event.select(
#     c("uuid"),
#     # c("source_fk"),
#     c("eq_fk"),
#     c("r"),
#     c("r0"),
#     c("x"),
#     c("x0"),
#     c("b"),
#     c("b0"),
#     c("g"),
#     c("g0"),
# )
# Add branch parameter to line_data
line_data = line_data.join(branch, left_on="uuid", right_on="eq_fk", how="left").drop(
    "uuid_right"
)

In [ ]:
node_id_list = (
    line_data.unpivot(on=["u_of_edge", "v_of_edge"], value_name="node_id")
    .unique("node_id", keep="first")["node_id"]
    .to_list()
)
node_id_to_nb_mapping = dict(zip(node_id_list, range(len(node_id_list))))
node_nb_to_id_mapping = dict(zip(range(len(node_id_list)), node_id_list))

line_data = line_data.with_columns(
    c(col).replace_strict(node_id_to_nb_mapping, default=None).alias(col)
    for col in ["u_of_edge", "v_of_edge"]
)
# ext_grid_nb = node_id_to_nb_mapping[ext_grid_id]

In [ ]:
# # from state_estimation_algorithm import StateEstimationData


# class StateEstimationData:
#     def __init__(self, changes_schema: dict):
#         self.line_data: pl.DataFrame
#         self.node_data: pl.DataFrame
#         self.changes_schema = changes_schema

#         self.generate_line_data()
#     ### TODO when code is ok
#     def generate_line_data(self):
#         self.line_data = self.changes_schema.resource.filter(
#             (c("concrete_class") == BRANCH) | (c("concrete_class") == SWITCH)
#         ).select(
#             c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type")
#         )
# stateestimationdata = StateEstimationData(changes_schema=changes_schema)

In [ ]:
ext_grid_id = "1"
n = 0.95
v_ext_grid_sq = 1.05

line_data: pl.DataFrame = pl.DataFrame(
    {
        "line_id": np.arange(1, 13).astype(str),
        "type": [BRANCH] * 10 + [TRANSFORMER] * 2,
        "u_of_edge": ["2", "2", "2", "4", "5", "6", "7", "10", "12", "13", "8", "9"],
        "v_of_edge": ["11", "1", "3", "1", "4", "4", "8", "9", "11", "11", "4", "8"],
        "n_transfo": [1.0] * 10 + [n**2, n**2],
        "x_pu": np.arange(1, 13) * 2e-3,
        "r_pu": np.arange(1, 13) * 1e-3,
        "b_pu": list(np.arange(1, 11) * -1e-3) + [0.001, 0.001],
        "g_pu": [0.0] * 10 + [0.001, 0.001],
    }
)

node_data: pl.DataFrame = pl.DataFrame(
    {
        "node_id": np.arange(1, 14).astype(str),
        "v_base": [400.0] * 7 + [200.0, 400.0, 400.0, 200.0, 100.0, 100.0],
        "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10]) * 1e-2,
        "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1]) * 1e-2,
    },
    strict=False,
)

np.set_printoptions(linewidth=200)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq

s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

In [ ]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/modified_cigre_network_lv.p")
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

In [ ]:
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow: DistFlow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

In [ ]:
line_res_pp = pl.DataFrame(
    {
        "node_id": list(net.line["to_bus"].astype(str)),
        "p_pp": list(net.res_line["p_from_mw"]),
        "q_pp": list(net.res_line["q_from_mvar"]),
    }
)

node_res_pp = pl.from_pandas(net.res_bus["vm_pu"], include_index=True).select(
    c("index").cast(pl.Utf8).alias("node_id"), "vm_pu"
)

result = (
    pl.DataFrame(
        {
            "v": np.sqrt(v),
            "p": np.real(s_flow),
            "q": np.imag(s_flow),
        }
    )
    .with_row_index(name="node_id")
    .with_columns(
        c("node_id")
        .replace_strict(distflow.node_nb_to_id_mapping, default=None)
        .alias("node_id")
    )
    .join(line_res_pp, on="node_id", how="left")
    .join(node_res_pp, on="node_id", how="left")
    .with_columns(
        ((c("v") - c("vm_pu")) / c("vm_pu")).abs().alias("diff_v"),
        ((c("p") - c("p_pp")) / c("p_pp")).abs().alias("diff_p"),
        ((c("q") - c("q_pp")) / c("p_pp")).abs().alias("diff_q"),
    )
)
print(result["diff_v", "diff_p", "diff_q"].max())

In [ ]:
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

In [ ]:
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

In [ ]:
line_data = line_data.group_by("u_of_edge", "v_of_edge").agg(
    c("name", "n_transfo", "type").first(),
    c("r_pu", "x_pu").first() / c("r_pu", "x_pu").count(),
    c("g_pu", "b_pu").first() * c("g_pu", "b_pu").count(),
)